In [ ]:
# default_exp losses.numpy

In [ ]:
#hide
%load_ext autoreload
%autoreload 2

# Metrics

> API details.

In [ ]:
#export
from math import sqrt
import numpy as np
from nixtla.losses.pytorch import divide_no_nan

# TODO: Think more efficient way of masking y_mask availability, without indexing (maybe 0s)

In [ ]:
# export
def metric_protections(y: np.ndarray, y_hat: np.ndarray, weights):
    assert (weights is None) or (np.sum(weights)>0), 'Sum of weights cannot be 0'
    assert (weights is None) or (len(weights)==len(y)), 'Wrong weight dimension'

In [ ]:
#export
def mae(y: np.ndarray, y_hat: np.ndarray, weights=None):
    """Calculates Mean Absolute Error.

    The mean absolute error 

    Parameters
    ----------
    y: numpy array
        actual test values
    y_hat: numpy array
        predicted values
    weights: numpy array
      weights for weigted average

    Return
    ------
    scalar: MAE
    """
    metric_protections(y, y_hat, weights)
    
    delta_y = np.abs(y - y_hat)
    if weights is not None:
      mae = np.average(delta_y[~np.isnan(delta_y)], weights=weights[~np.isnan(delta_y)])
    else:
      mae = np.nanmean(delta_y)
    return mae

In [ ]:
#export
def mse(y: np.ndarray, y_hat: np.ndarray, weights=None) -> float:
    """Calculates Mean Squared Error.
    MSE measures the prediction accuracy of a
    forecasting method by calculating the squared deviation
    of the prediction and the true value at a given time and
    averages these devations over the length of the series.
    
    Parameters
    ----------
    y: numpy array
        actual test values
    y_hat: numpy array
        predicted values 
    weights: numpy array
      weights for weigted average
        
    Returns
    -------
    scalar: MSE
    """
    metric_protections(y, y_hat, weights)

    delta_y = np.square(y - y_hat)
    if weights is not None:
      mse = np.average(delta_y[~np.isnan(delta_y)], weights=weights[~np.isnan(delta_y)])
    else:
      mse = np.nanmean(delta_y)
    return mse

In [ ]:
#export
def rmse(y: np.ndarray, y_hat: np.ndarray, weights=None) -> float:
    """Calculates Root Mean Squared Error.
    RMSE measures the prediction accuracy of a
    forecasting method by calculating the squared deviation
    of the prediction and the true value at a given time and
    averages these devations over the length of the series.
    Finally the RMSE will be in the same scale
    as the original time series so its comparison with other
    series is possible only if they share a common scale.
    
    Parameters
    ----------
    y: numpy array
      actual test values
    y_hat: numpy array
      predicted values    
    weights: numpy array
      weights for weigted average      
      
    Returns
    -------
    scalar: RMSE
    """

    return np.sqrt(mse(y, y_hat, weights))

In [ ]:
#export
def mape(y: np.ndarray, y_hat: np.ndarray, weights=None) -> float:
    #TODO: weights no hace nada
    """Calculates Mean Absolute Percentage Error.
    MAPE measures the relative prediction accuracy of a
    forecasting method by calculating the percentual deviation
    of the prediction and the true value at a given time and
    averages these devations over the length of the series.
    
    Parameters
    ----------
    y: numpy array
      actual test values
    y_hat: numpy array
      predicted values
    weights: numpy array
      weights for weigted average
 
    Returns
    -------
    scalar: MAPE
    """
    metric_protections(y, y_hat, weights)
        
    delta_y = np.abs(y - y_hat)
    scale = np.abs(y)
    mape = divide_no_nan(delta_y, scale)
    mape = np.average(mape, weights=weights)
    mape = 100 * mape
    return mape

In [ ]:
#export
def smape(y: np.ndarray, y_hat: np.ndarray,
          y_mask=None, weights=None) -> float:
    """Calculates Symmetric Mean Absolute Percentage Error.
    
    SMAPE measures the relative prediction accuracy of a
    forecasting method by calculating the relative deviation
    of the prediction and the true value scaled by the sum of the
    absolute values for the prediction and true value at a
    given time, then averages these devations over the length
    of the series. This allows the SMAPE to have bounds between
    0% and 200% which is desireble compared to normal MAPE that
    may be undetermined.
    
    Parameters
    ----------
    y: numpy array
      actual test values
    y_hat: numpy array
      predicted values
    y_mask: numpy array
      optional mask, 1 keep 0 omit      
    weights: numpy array
      weights for weigted average
    
    Returns
    -------
    scalar: SMAPE
    """
    metric_protections(y, y_hat, weights)
        
    delta_y = np.abs(y - y_hat)
    scale = np.abs(y) + np.abs(y_hat)
    smape = divide_no_nan(delta_y, scale)
    smape = 200 * np.average(smape, weights=weights)

    assert smape <= 200, 'SMAPE should be lower than 200'
    return smape

In [ ]:
#export
def pinball_loss(y: np.ndarray, y_hat: np.ndarray, tau: float=0.5, weights=None) -> np.ndarray:
    """Calculates the Pinball Loss.

    The Pinball loss measures the deviation of a quantile forecast.
    By weighting the absolute deviation in a non symmetric way, the
    loss pays more attention to under or over estimation.
    A common value for tau is 0.5 for the deviation from the median.

    Parameters
    ----------
    y: numpy array
      actual test values
    y_hat: numpy array of len h (forecasting horizon)
      predicted values   
    weights: numpy array
      weights for weigted average      
    tau: float
      Fixes the quantile against which the predictions are compared.
    Return
    ------
    return: pinball_loss
    """
    metric_protections(y, y_hat, weights)

    delta_y = y - y_hat
    pinball = np.maximum(tau * delta_y, (tau - 1) * delta_y)

    if weights is not None:
      pinball = np.average(pinball[~np.isnan(pinball)], weights=weights[~np.isnan(pinball)])
    else:
      pinball = np.nanmean(pinball)
    return pinball

In [ ]:
# export
def accuracy_logits(y: np.ndarray, y_hat: np.ndarray, weights=None, thr=0.5) -> np.ndarray:
    """Calculates the Accuracy.

    Parameters
    ----------
    y: numpy array
      actual test values
    y_hat: numpy array of len h (forecasting horizon)
      predicted values
    weights: numpy array
      weights for weigted average
    Return
    ------
    return accuracy
    """
    metric_protections(y, y_hat, weights)

    y_hat = ((1/(1 + np.exp(-y_hat))) > thr) * 1
    correct = (y==y_hat)[~np.isnan(y)]
    if weights is not None:
      accuracy = np.average(correct, weights=weights[~np.isnan(y)]) * 100
    else:
      accuracy = np.nanmean(correct) * 100
    return accuracy

# TEST NUMPY EVALUATION METRICS

In [ ]:
y = np.array([1,1,1,0,0,0,0,0,1, np.nan])
y_mask = np.array([1,1,1,1,1,1,1,1,2,0])
y_hat = np.array([1,2,3,-4,-5,-6,-7,-8,-9,-10])

print(accuracy_logits(y=y, y_hat=y_hat, weights=y_mask))
print(accuracy_logits(y=y, y_hat=y_hat))

80.0
88.88888888888889


In [ ]:
print(mae(y=y, y_hat=y_hat, weights=y_mask))
print(mae(y=y, y_hat=y_hat))

5.3
4.777777777777778


In [ ]:
len(y)

11